In [1]:
# Import libaries
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import requests
from bs4 import BeautifulSoup
import time

In [195]:
def overall_season(year):
     # Set up our url to webscrape from using the year
    overall_season_url = "https://www.basketball-reference.com/leagues/NBA_" + str(year) + ".html"
    # Utilize requests and BeautifulSoup libraries
    res = requests.get(overall_season_url)
    soup = BeautifulSoup(res.content, "lxml")
    
    teams = pd.DataFrame()
    
    conferences = {
        "Eastern" : "E",
        "Western" : "W"
    }

    for conference, symbol in conferences.items():

        table_title = "divs_standings_" + symbol

        table = soup.find("table", {"id" : table_title})
        
        full_conference = []
        team_basics = ["Year","Conference", "Team Name", "Team Abbreviation"]
        for header in table.find_all("tbody"):
            for row in header.find_all("th", {"class" : "left"}):
                team_name = row.find('a').text
                team_url = row.find('a').attrs["href"].split("/")
                team_abbreviation = team_url[2]
                conference_basics = dict(zip(team_basics,[year, conference, team_name, team_abbreviation]))
                full_conference.append(conference_basics)
        
        full_conference_df = pd.DataFrame(full_conference)
        
        team_cols = []
        for header in table.find_all("thead"):
            for row in header.find_all("th", {"class" : "poptip right"}):
                team_cols.append(row.text)
            team_cols.append("GB")
            order = [0, 1, 2, 6, 3, 4, 5]
            team_cols = [team_cols[i] for i in order]
        
        
        team_standings_stats = []
        for body in table.find_all("tbody"):
            for row in body.find_all("tr", {"class" : "full_table"}):
                team_standings = dict(zip(team_cols,[stat.text for stat in row.find_all("td")]))
                team_standings_stats.append(team_standings)
        
        team_standings_df = pd.DataFrame(team_standings_stats)
        
        team_standings_df = team_standings_df.loc[:, ['W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS']]
        
        conference_standings = pd.concat([full_conference_df, team_standings_df], axis = 1)
        
        teams = teams.append(conference_standings)
        
    teams.reset_index(drop = True, inplace = True)
    
    teams.rename(columns = {
        "W"    : "Wins",
        "L"    : "Losses",
        "W/L%" : "Win/Loss Percentage",
        "GB"   : "Games Back",
        "PS/G" : "Points Per Game",
        "PA/G" : "Opponent Points Per Game",
        "SRS"  : "Simple Rating System"
    }, inplace = True)
    
    return teams

In [199]:
test_df = overall_season(1985)

test_df

,Year,Conference,Team Name,Team Abbreviation,Wins,Losses,Win/Loss Percentage,Games Back,Points Per Game,Opponent Points Per Game,Simple Rating System
0,1985,Eastern,Boston Celtics,BOS,63,19,.768,—,114.8,108.1,6.47
1,1985,Eastern,Philadelphia 76ers,PHI,58,24,.707,5.0,112.9,108.8,4.17
2,1985,Eastern,New Jersey Nets,NJN,42,40,.512,21.0,109.5,109.2,0.64
3,1985,Eastern,Washington Bullets,WSB,40,42,.488,23.0,105.5,105.8,0.15
4,1985,Eastern,New York Knicks,NYK,24,58,.293,39.0,105.2,109.8,-4.09
5,1985,Eastern,Milwaukee Bucks,MIL,59,23,.720,—,110.9,104.0,6.69
6,1985,Eastern,Detroit Pistons,DET,46,36,.561,13.0,116.0,113.5,2.73
7,1985,Eastern,Chicago Bulls,CHI,38,44,.463,21.0,108.7,109.6,-0.50
8,1985,Eastern,Cleveland Cavaliers,CLE,36,46,.439,23.0,108.6,111.3,-2.27
9,1985,Eastern,Atlanta Hawks,ATL,34,48,.415,25.0,106.6,108.1,-1.14


In [146]:
for name in test_df["Team Abbreviation"]:
    print (name)

MIL
TOR
PHI
BOS
IND
BRK
ORL
DET
CHO
MIA
WAS
ATL
CHI
CLE
NYK
GSW
DEN
POR
HOU
UTA
OKC
SAS
LAC
SAC
LAL
MIN
MEM
NOP
DAL
PHO


### Season Schedule Dataframe
This function webscrapes from basketballreference.com to get the full schedule of any team from any year. It collects relevant information that can then be converted into game data from another function. The input parameters are the team's abbreviation, the year, and specificying if the user wants data from the regular season or playoffs.

In [128]:
def season_schedule(team_abbreviation, point_in_season, dataframe):
    
    for i in range(0, len(dataframe["Team Abbreviation"])):
        if team_abbreviation == dataframe["Team Abbreviation"][i]:
            team = dataframe["Team Name"][i]
    
    # Set up our url to webscrape from using the team's abbreviation and year
    season_url = "https://www.basketball-reference.com/teams/" + team_abbreviation + "/" + str(dataframe["Year"][0]) + "_games.html"
    # Utilize requests and BeautifulSoup libraries
    res = requests.get(season_url)
    soup = BeautifulSoup(res.content, "lxml")
    
    # Specify the tables to pull from - either regular season or playoffs
    if point_in_season.lower() == "regular season":
        game_type = "games"
    elif point_in_season.lower() == "playoffs":
        game_type = "games_playoffs"
    
    # Set up the table
    table = soup.find("table", {"id" : game_type})
    
    # create a list of column headers
    games_cols = []
    for header in table.find_all("thead"):
        for row in header.find_all("th", {"class" : "poptip sort_default_asc left"}):
            games_cols.append(row.text)
    games_cols.append("Game")
    games_cols.append("Location")
    games_cols.append("Result")
    games_cols.append("Overtime")
    games_cols.append("Wins")
    games_cols.append("Losses")
    games_cols.append("Time Period")
    games_cols.append("Opponent Abbreviation")
    
    # webscrape for content from the table
    # create an empty list to append all rows of data as dictionaries to be converted into a dataframe
    game_review_list = []
    # looking within the body of the table,
    for body in table.find_all("tbody"):
        # iterate through each row - excluding rows with classes, which have no relevant data
        for row in body.find_all("tr", {"class" : None}):
            game = row.find("th", {"data-stat" : "g"}).text # game number in the season
            date = (row.find("td", {"data-stat" : "date_game"}).attrs["csk"]) # date of the game
            opponent = row.find("td", {"data-stat" : "opp_name"}).text # opponent to the team
            location = row.find("td", {"data-stat" : "game_location"}).text # Home or Away
            result = row.find("td", {"data-stat" : "game_result"}).text # Win or loss
            overtime = row.find("td", {"data-stat" : "overtimes"}).text # whether overtime was required
            wins = row.find("td", {"data-stat" : "wins"}).text # season wins record
            losses = row.find("td", {"data-stat" : "losses"}).text # season losses record
            stage = point_in_season.title() # regular season or playoff
            # zip the columns with the data into a dictionary
            
            for i in range(0, len(dataframe["Team Name"])):
                if opponent == dataframe["Team Name"][i]:
                    abbr = dataframe["Team Abbreviation"][i]
            
            game_review = dict(zip(games_cols,[date, opponent, game, location, result, 
                                               overtime, wins, losses, stage, abbr]))
            # add each dictionary to the original list
            game_review_list.append(game_review)
    # create a dataframe from our final list        
    game_review_df = pd.DataFrame(game_review_list)
    # add a few columns to our list
    
    game_review_df["Team"] = team
    game_review_df["Team Abbreviation"] = team_abbreviation
    game_review_df["Season"] = dataframe["Year"][0] # what year this season was
    # data from game location needs to be converted to human readable "Home" and "Away"
    game_review_df["Location"] = game_review_df["Location"].map({"" : "Home", "@" : "Away"})
    # reorder the dataframe
    game_review_df = game_review_df.loc[:, ["Team", "Team Abbreviation", "Season", "Time Period", "Game", "Date",
                                            "Opponent", "Opponent Abbreviation", 
                                            "Location", "Result", "Overtime", "Wins", "Losses"]]
    # return season dataframe
    return game_review_df

In [133]:
test_season_df = season_schedule("MIL", "Regular Season", test_df)
test_season_df

,Team,Team Abbreviation,Season,Time Period,Game,Date,Opponent,Opponent Abbreviation,Location,Result,Overtime,Wins,Losses
0,Milwaukee Bucks,MIL,2019,Regular Season,1,2018-10-17,Charlotte Hornets,CHO,Away,W,,1,0
1,Milwaukee Bucks,MIL,2019,Regular Season,2,2018-10-19,Indiana Pacers,IND,Home,W,,2,0
2,Milwaukee Bucks,MIL,2019,Regular Season,3,2018-10-22,New York Knicks,NYK,Home,W,,3,0
3,Milwaukee Bucks,MIL,2019,Regular Season,4,2018-10-24,Philadelphia 76ers,PHI,Home,W,,4,0
4,Milwaukee Bucks,MIL,2019,Regular Season,5,2018-10-26,Minnesota Timberwolves,MIN,Away,W,,5,0
5,Milwaukee Bucks,MIL,2019,Regular Season,6,2018-10-27,Orlando Magic,ORL,Home,W,,6,0
6,Milwaukee Bucks,MIL,2019,Regular Season,7,2018-10-29,Toronto Raptors,TOR,Home,W,,7,0
7,Milwaukee Bucks,MIL,2019,Regular Season,8,2018-11-01,Boston Celtics,BOS,Away,L,,7,1
8,Milwaukee Bucks,MIL,2019,Regular Season,9,2018-11-04,Sacramento Kings,SAC,Home,W,,8,1
9,Milwaukee Bucks,MIL,2019,Regular Season,10,2018-11-06,Portland Trail Blazers,POR,Away,L,,8,2


### Game Data Function
This function is designed to webscrape individual game data for one team from any game in any season. It collects data from basic game stats and advanced stats and collects them all together, with the overall team data as well. It then creates a dataframe from those tables. It requires the url from the game from basketballreference.com, the team's abbreviation, the season, date of the game, game number of the season, whether this was a home or away game, whether this was the regular season or playoffs, whether the game required overtime, and whether the team won or lost.

In [154]:
def get_game_stats(url, team_abbreviation, date, dataframe):
    # Utilize requests and BeautifulSoup libraries
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "lxml")

    # Identify our two tables, basic and advanced data
    table = soup.find("table", {"id" : "box-" + team_abbreviation + "-game-basic"})
    table2 = soup.find("table", {"id" : "box-" + team_abbreviation + "-game-advanced"})

    # create a dataframe of our players
    # start with an empty list
    players_list = []
    # iterate through the basic stats table
#     for body in table.find_all("tbody"):
    for row in table.find_all("th", {"class" : "left"}):
        players_list.append(row.text) # add each player's name
    # create a dataframe with one column that is just all the players    
    players_df = pd.DataFrame(players_list, columns = ["Player"])
    # add some rows to this dataframe
    players_df["Team Abbreviation"] = team_abbreviation # add team abbreviation

    
    players_df["Season"] = dataframe["Season"][0] # Add the season
    players_df["Date"] = date # add the date of the game
    
    for i in range(0, dataframe.shape[0]):
        if date == dataframe["Date"][i]:
            players_df["Overtime"] = dataframe["Overtime"][i]
            
            if team_abbreviation == dataframe["Opponent Abbreviation"][i]:
                
                players_df["Team"] = dataframe["Opponent"][i]
                
                if dataframe["Location"][i] == "Home":
                    players_df["Location"] = "Away"
                else:
                    players_df["Location"] = "Home"
                    
                if dataframe["Result"][i] == "W":
                    players_df["Result"] = "L"
                else:
                    players_df["Result"] = "W"

            else:
                players_df["Team"] = dataframe["Team"][0]
                players_df["Location"] = dataframe["Location"][i]
                players_df["Result"] = dataframe["Result"][i]
        
    players_df["Time Period"] = dataframe["Time Period"][0] # add column for regular season or playoffs
 

    # reorder the dataframe to a more human readable version
    players_df = players_df.loc[:, ["Season","Time Period", "Date", "Team", "Team Abbreviation", 
                                    "Location", "Overtime", "Result", "Player"]]
    
    # create another dataframe of the game stats to be combined with our other dataframe
    # Identify our columns from the basic table
    # create an empty list
    stats_cols = []
    # iterate through all our rows that are column headers
    for row in table.find_all("th", {"class" : "poptip center"}):
        stats_cols.append(row.text) # add column headers to the list

    # combine our table body into a list of zipped dictionaries
    # add empty list
    stats_box_list = []
    # iterate through the body of the table
    for body in table.find_all("tbody"):
        
        for row in body.find_all("tr"):
            stats = dict(zip(stats_cols,[stat.text for stat in row.find_all("td")]))
            stats_box_list.append(stats)
            

    # Create a dataframe of the first table
    stats_df = pd.DataFrame(stats_box_list).dropna().reset_index(drop = True)
    
    players_df_basic_stats = pd.concat([players_df, stats_df], axis = 1)
    players_df_basic_stats.drop(index = (players_df_basic_stats.loc[players_df_basic_stats["MP"].isna(),:].index),
                                inplace = True)
    
    
    team_df = pd.DataFrame(["Team Totals"], columns = ["Player"])
        # add some rows to this dataframe

    team_df["Team Abbreviation"] = team_abbreviation # add team abbreviation
    team_df["Season"] = players_df["Season"][0]
    team_df["Time Period"] = players_df["Time Period"][0]
    team_df["Date"] = players_df["Date"][0]
    team_df["Team"] = players_df["Team"][0]
    team_df["Location"] = players_df["Location"][0]
    team_df["Overtime"] = players_df["Overtime"][0]
    team_df["Result"] = players_df["Result"][0]
    
    # reorder the dataframe to a more human readable version
    team_df = team_df.loc[:, ["Season","Time Period", "Date", "Team", "Team Abbreviation",
                                    "Location", "Overtime", "Result", "Player"]]
    
    team_stats_box_list = []
    # add in the bottom row of team overall data
    for foot in table.find_all("tfoot"):
        for row in foot.find_all("tr"):
            team_stats = dict(zip(stats_cols,[stat.text for stat in row.find_all("td")]))
            team_stats_box_list.append(team_stats)
            
    team_stats_df = pd.DataFrame(team_stats_box_list)
    
    team_df_basic_stats = pd.concat([team_df, team_stats_df], axis = 1)
    
    basic_stats_table = pd.concat([players_df_basic_stats, team_df_basic_stats], axis = 0)
    basic_stats_table.reset_index(inplace = True, drop = True)
    
    
    # Collect our column names
    adv_stats_cols = []
    for row in table2.find_all("th", {"class" : "poptip center"}):
        adv_stats_cols.append(row.text)

    # collect our advanced stats from 

    for row in table2.find_all("th", {"class" : "poptip sort_default_asc center"}):
        adv_stats_cols.append(row.text)

    for row in table2.find_all("th", {"class" : "poptip right"}):
        adv_stats_cols.append(row.text)

    order = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14, 11, 12, 15, 16, 13]
    adv_stats_cols = [adv_stats_cols[i] for i in order]
    adv_stats_cols = adv_stats_cols[:-1]

    # Collect our stats from the advaced table and create a zipped dictionary
    stats_box_list = []
    for body in table2.find_all("tbody"):
        for row in body.find_all("tr"):
            stats = dict(zip(adv_stats_cols,[stat.text for stat in row.find_all("td")]))
            stats_box_list.append(stats)

    for foot in table2.find_all("tfoot"):
        for row in foot.find_all("tr"):
            stats = dict(zip(adv_stats_cols,[stat.text for stat in row.find_all("td")]))
            stats_box_list.append(stats)

    # create a dataframe of the advanced stats that drops the repeated minutes played row
    advanced_stats_df = pd.DataFrame(stats_box_list).dropna().reset_index(drop = True).drop(columns = "MP")

    # Concatenate all 3 data sets into one
    box_stats_df = pd.concat([basic_stats_table, advanced_stats_df], axis = 1)
    
    return box_stats_df

In [155]:
# def get_game_stats(url, team_abbreviation, date, dataframe):
get_game_stats("https://www.basketball-reference.com/boxscores/201902060MIL.html","WAS", "2019-02-06", test_season_df)

,Season,Time Period,Date,Team,Team Abbreviation,Location,Overtime,Result,Player,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,2019,Regular Season,2019-02-06,Washington Wizards,WAS,Away,,L,Bradley Beal,39:08,12,22,.545,2,8,.250,4,5,.800,1,7,8,4,3,1,8,5,30,.620,.591,.364,.227,3.6,21.5,13.3,14.7,3.3,1.9,24.8,31.8,96,127,-4.1
1,2019,Regular Season,2019-02-06,Washington Wizards,WAS,Away,,L,Tomáš Satoranský,37:37,5,10,.500,3,4,.750,3,4,.750,0,7,7,10,1,0,4,1,16,.680,.650,.400,.400,0.0,22.3,12.1,30.7,1.1,0.0,25.4,16.2,123,134,0.8
2,2019,Regular Season,2019-02-06,Washington Wizards,WAS,Away,,L,Jeff Green,37:08,6,10,.600,5,9,.556,5,5,1.000,2,2,4,3,0,1,4,4,22,.902,.850,.900,.500,7.6,6.5,7.0,9.6,0.0,2.1,24.7,16.8,135,137,2.1
3,2019,Regular Season,2019-02-06,Washington Wizards,WAS,Away,,L,Trevor Ariza,31:53,1,6,.167,1,3,.333,2,2,1.000,1,2,3,8,2,0,3,2,5,.363,.250,.500,.333,4.4,7.5,6.1,25.9,2.7,0.0,30.4,12.0,99,132,-2.9
4,2019,Regular Season,2019-02-06,Washington Wizards,WAS,Away,,L,Thomas Bryant,31:45,12,19,.632,1,3,.333,1,1,1.000,3,11,14,4,0,2,0,2,26,.669,.658,.158,.053,13.3,41.6,28.6,20.3,0.0,4.8,0.0,23.6,146,130,8.3
5,2019,Regular Season,2019-02-06,Washington Wizards,WAS,Away,,L,Sam Dekker,20:57,5,10,.500,2,4,.500,0,0,,0,2,2,1,0,0,2,4,12,.600,.600,.400,.000,0.0,11.5,6.2,6.3,0.0,0.0,16.7,22.1,96,137,-8.8
6,2019,Regular Season,2019-02-06,Washington Wizards,WAS,Away,,L,Jordan McRae,19:19,4,7,.571,1,2,.500,0,0,,0,0,0,0,0,1,2,1,9,.643,.643,.286,.000,0.0,0.0,0.0,0.0,0.0,3.9,22.2,18.0,89,136,-7.3
7,2019,Regular Season,2019-02-06,Washington Wizards,WAS,Away,,L,Chasson Randle,15:20,2,7,.286,1,3,.333,1,2,.500,1,1,2,4,0,0,1,4,6,.381,.357,.429,.286,9.2,7.8,8.5,30.0,0.0,0.0,11.3,22.4,101,137,-11.2
8,2019,Regular Season,2019-02-06,Washington Wizards,WAS,Away,,L,John Jenkins,6:53,1,1,1.000,1,1,1.000,0,0,,0,0,0,0,0,0,0,0,3,1.500,1.500,1.000,.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.6,300,140,4.5
9,2019,Regular Season,2019-02-06,Washington Wizards,WAS,Away,,L,Team Totals,240,48,92,.522,17,37,.459,16,19,.842,8,32,40,34,6,5,24,23,129,.643,.614,.402,.207,23.5,80.0,54.1,70.8,5.4,7.9,19.3,100.0,116.2,133.3,


### All Seasons' Stats Function

In [187]:
def season_stats(list_of_years, stages):
    base_url = "https://www.basketball-reference.com/boxscores/"
    
    final_df = pd.DataFrame()
    for year in list_of_years:
        season_df = overall_season(year)
        for stage in stages:
            for team_abbr in season_df["Team Abbreviation"]:
                schedule_df = season_schedule(team_abbr, stage, season_df)

                for i in range(len(schedule_df.index)):
                    date = schedule_df["Date"][i].replace("-", "")
                    if schedule_df["Location"][i] == "Home":
                        city = schedule_df["Team Abbreviation"][i]
                    else:
                        city = schedule_df["Opponent Abbreviation"][i]

                    game_url = base_url + date + "0" + city + ".html"
                    # def get_game_stats(url, team_abbreviation, date, dataframe):
                    game_df = pd.concat([get_game_stats(game_url,
                                                        schedule_df["Team Abbreviation"][i],
                                                        schedule_df["Date"][i],
                                                        schedule_df),
                                         get_game_stats(game_url, 
                                                        schedule_df["Opponent Abbreviation"][i], 
                                                        schedule_df["Date"][i], 
                                                        schedule_df)])

                    final_df = final_df.append(game_df)
                    if i % 20 == 0 and i !=0:
                        message = (
                            f'Collecting statistics from the {i +1}st game of the {year} season for '
                            f'the {schedule_df["Team"][i]} when they played the {schedule_df["Opponent"][i]} on '
                            f'{schedule_df["Date"][i]} where they got the {schedule_df["Result"][i]}.'
                        )
                        print (message)
                    if i == 81:
                        final_df.to_csv("../data/teams_csv", index = False)
                    time.sleep(2)
    final_df['Date'] = pd.to_datetime(final_df['Date'])
    final_df.reset_index(drop = True, inplace = True)
    
    return final_df

In [188]:
nba_2018_2019_seasons = season_stats([2018, 2019], ["Regular Season"])
nba_2018_2019_seasons.head(100)

,Season,Time Period,Date,Team,Team Abbreviation,Location,Overtime,Result,Player,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,2018,Regular Season,2017-10-19,Toronto Raptors,TOR,Home,,W,DeMar DeRozan,32:03,2,9,.222,0,0,,7,8,.875,1,5,6,5,2,1,5,1,11,.439,.222,.000,.889,3.6,16.3,10.2,20.8,3.0,2.6,28.5,23.4,86,97,-5.3
1,2018,Regular Season,2017-10-19,Toronto Raptors,TOR,Home,,W,Kyle Lowry,31:20,4,7,.571,1,3,.333,3,3,1.000,0,4,4,9,1,0,4,1,12,.721,.643,.429,.429,0.0,13.3,7.0,41.9,1.5,0.0,32.5,16.8,121,103,2.3
2,2018,Regular Season,2017-10-19,Toronto Raptors,TOR,Home,,W,Jonas Valančiūnas,30:30,9,17,.529,0,1,.000,5,6,.833,8,7,15,2,2,0,1,4,23,.586,.529,.059,.353,30.0,23.9,26.8,12.7,3.2,0.0,4.8,28.9,136,96,4.3
3,2018,Regular Season,2017-10-19,Toronto Raptors,TOR,Home,,W,Norman Powell,24:35,5,11,.455,3,6,.500,2,2,1.000,0,4,4,1,1,0,2,2,15,.631,.591,.545,.182,0.0,17.0,8.9,6.7,2.0,0.0,14.4,24.1,111,101,0.0
4,2018,Regular Season,2017-10-19,Toronto Raptors,TOR,Home,,W,Serge Ibaka,24:14,3,8,.375,2,4,.500,0,0,,1,3,4,0,0,2,0,2,8,.500,.500,.500,.000,4.7,12.9,9.0,0.0,0.0,7.0,0.0,14.1,113,101,-2.0
5,2018,Regular Season,2017-10-19,Toronto Raptors,TOR,Home,,W,Delon Wright,23:22,4,6,.667,0,1,.000,5,5,1.000,1,1,2,5,1,0,0,0,13,.793,.667,.167,.833,4.9,4.5,4.7,33.4,2.1,0.0,0.0,15.0,185,104,11.9
6,2018,Regular Season,2017-10-19,Toronto Raptors,TOR,Home,,W,C.J. Miles,20:01,7,12,.583,6,9,.667,2,2,1.000,0,5,5,0,0,0,0,4,22,.854,.833,.750,.167,0.0,26.1,13.6,0.0,0.0,0.0,0.0,27.5,165,102,15.1
7,2018,Regular Season,2017-10-19,Toronto Raptors,TOR,Home,,W,Jakob Poeltl,17:30,1,2,.500,0,0,,0,0,,0,5,5,0,1,0,3,3,2,.500,.500,.000,.000,0.0,29.8,15.6,0.0,2.7,0.0,60.0,12.2,34,95,-12.0
8,2018,Regular Season,2017-10-19,Toronto Raptors,TOR,Home,,W,OG Anunoby,17:14,3,6,.500,1,3,.333,2,2,1.000,2,1,3,2,0,0,0,1,9,.654,.583,.500,.333,13.3,6.1,9.5,18.2,0.0,0.0,0.0,17.1,158,108,5.5
9,2018,Regular Season,2017-10-19,Toronto Raptors,TOR,Home,,W,Fred VanVleet,12:59,1,5,.200,0,2,.000,0,0,,0,1,1,2,1,1,2,1,2,.200,.200,.400,.000,0.0,8.0,4.2,20.9,3.7,6.5,28.6,23.0,49,95,-11.2


In [96]:
.rename(columns = {
    "TS%"  : "True Shooting Percentage",
    "eFG%" : "Effective Field Goal Percentage",
    "3PAr" : "3 Point Attempt Rate",
    "FTr"  : "Free Throw Attempt Rate",
    "ORB%" : "Offensive Rebound Percentage",
    "DRB%" : "Defensive Rebound Percentage",
    "TRB%" : "Total Rebound Percentage",
    "AST%" : "Assist Percentage",
    "STL%" : "Steal Percentage",
    "BLK%" : "Block Percentage",
    "TOV%" : "Turnover Percentage",
    "USG%" : "Usage Percentage",
    "ORtg" : "Offensive Rating",
    "DRtg" : "Defensive Rating",
    "BPM"  : "Box Plus/Minus",
    "MP"   : "Minutes Played",
    "FG"   : "Field Goals",
    "FGA"  : "Field Goal Attempts",
    "FG%"  : "Field Goal Percentage",
    "3P"   : "3-Point Field Goals",
    "3PA"  : "3-Point Attempts",
    "3P%"  : "3-Point Percentage",
    "FT"   : "Free Throws",
    "FTA"  : "Free Throw Attempts",
    "FT%"  : "Free Throw Percentage",
    "ORB"  : "Offensive Rebounds",
    "DRB"  : "Defensive Rebounds",
    "TRB"  : "Total Rebounds",
    "AST"  : "Assists",
    "STL"  : "Steals",
    "BLK"  : "Blocks",
    "TOV"  : "Turnovers",
    "PF"   : "Personal Fouls",
    "PTS"  : "Points"
    }, inplace = True)
mj_seasons.head(15)

,Season,Time Period,Game,Date,Team Initialism,Team,Location,Overtime,Result,Player,Minutes Played,Field Goals,Field Goal Attempts,Field Goal Percentage,3-Point Field Goals,3-Point Attempts,3-Point Percentage,Free Throws,Free Throw Attempts,Free Throw Percentage,Offensive Rebounds,Defensive Rebounds,Total Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points,True Shooting Percentage,Effective Field Goal Percentage,3 Point Attempt Rate,Free Throw Attempt Rate,Offensive Rebound Percentage,Defensive Rebound Percentage,Total Rebound Percentage,Assist Percentage,Steal Percentage,Block Percentage,Turnover Percentage,Usage Percentage,Offensive Rating,Defensive Rating,Box Plus/Minus
0,1985,Regular Season,1,1984-10-26,CHI,Chicago Bulls,Home,,W,Michael Jordan,40:00,5,16,.313,0,0,,6,7,.857,1,5,6,7,2,4,5,2,16,.419,.313,.000,.438,3.1,11.3,7.8,20.0,2.3,5.5,20.8,24.4,86,85,0.8
1,1985,Regular Season,1,1984-10-26,CHI,Chicago Bulls,Home,,W,Orlando Woolridge,39:00,13,19,.684,0,0,,2,5,.400,2,7,9,3,0,1,0,1,28,.660,.684,.000,.263,6.3,16.3,12.0,11.5,0.0,1.4,0.0,22.0,138,90,8.5
2,1985,Regular Season,1,1984-10-26,CHI,Chicago Bulls,Home,,W,Ennis Whatley,37:00,5,8,.625,0,0,,1,1,1.000,1,3,4,10,2,0,4,2,11,.652,.625,.000,.125,3.3,7.3,5.6,31.3,2.4,0.0,32.2,13.6,110,89,1.9
3,1985,Regular Season,1,1984-10-26,CHI,Chicago Bulls,Home,,W,Steve Johnson,35:00,9,14,.643,0,0,,0,1,.000,5,4,9,3,2,1,2,2,18,.623,.643,.000,.071,17.6,10.4,13.4,11.5,2.6,1.6,12.2,19.0,120,87,6.2
4,1985,Regular Season,1,1984-10-26,CHI,Chicago Bulls,Home,,W,Caldwell Jones,29:00,0,5,.000,0,1,.000,0,0,,0,5,5,5,0,0,0,4,0,.000,.000,.200,.000,0.0,15.6,9.0,17.2,0.0,0.0,0.0,7.0,57,90,-3.2
5,1985,Regular Season,1,1984-10-26,CHI,Chicago Bulls,Home,,W,Quintin Dailey,28:00,12,18,.667,0,0,,1,1,1.000,1,1,2,1,1,0,7,3,25,.678,.667,.000,.056,4.4,3.2,3.7,6.3,1.6,0.0,27.5,36.8,90,93,-3.4
6,1985,Regular Season,1,1984-10-26,CHI,Chicago Bulls,Home,,W,Jawann Oldham,13:00,0,1,.000,0,1,.000,0,0,,0,3,3,0,0,3,1,1,0,.000,.000,1.000,.000,0.0,20.9,12.0,0.0,0.0,12.7,50.0,6.2,0,81,-1.4
7,1985,Regular Season,1,1984-10-26,CHI,Chicago Bulls,Home,,W,Sidney Green,9:00,2,4,.500,0,0,,0,0,,1,6,7,0,0,0,3,1,4,.500,.500,.000,.000,13.7,60.4,40.6,0.0,0.0,0.0,42.9,31.5,58,71,-13.9
8,1985,Regular Season,1,1984-10-26,CHI,Chicago Bulls,Home,,W,Dave Corzine,6:00,1,2,.500,0,0,,0,0,,0,1,1,0,0,0,1,2,2,.500,.500,.000,.000,0.0,15.1,8.7,0.0,0.0,0.0,33.3,20.3,62,90,-15.7
9,1985,Regular Season,1,1984-10-26,CHI,Chicago Bulls,Home,,W,Rod Higgins,2:00,1,1,1.000,1,1,1.000,0,0,,0,0,0,0,0,0,0,0,3,1.500,1.500,1.000,.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.3,300,98,43.8


In [97]:
mj_seasons["Overtime"].value_counts()

       28556
OT      1336
2OT      155
3OT       68
Name: Overtime, dtype: int64

In [98]:
mj_seasons["Overtime"] = mj_seasons["Overtime"].map({""   : 0,
                                                    "OT"  : 1,
                                                    "2OT" : 2,
                                                    "3OT" : 3,
                                                    "4OT" : 4})

mj_seasons.head()

,Season,Time Period,Game,Date,Team Initialism,Team,Location,Overtime,Result,Player,Minutes Played,Field Goals,Field Goal Attempts,Field Goal Percentage,3-Point Field Goals,3-Point Attempts,3-Point Percentage,Free Throws,Free Throw Attempts,Free Throw Percentage,Offensive Rebounds,Defensive Rebounds,Total Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points,True Shooting Percentage,Effective Field Goal Percentage,3 Point Attempt Rate,Free Throw Attempt Rate,Offensive Rebound Percentage,Defensive Rebound Percentage,Total Rebound Percentage,Assist Percentage,Steal Percentage,Block Percentage,Turnover Percentage,Usage Percentage,Offensive Rating,Defensive Rating,Box Plus/Minus
0,1985,Regular Season,1,1984-10-26,CHI,Chicago Bulls,Home,0,W,Michael Jordan,40:00,5,16,.313,0,0,,6,7,.857,1,5,6,7,2,4,5,2,16,.419,.313,.000,.438,3.1,11.3,7.8,20.0,2.3,5.5,20.8,24.4,86,85,0.8
1,1985,Regular Season,1,1984-10-26,CHI,Chicago Bulls,Home,0,W,Orlando Woolridge,39:00,13,19,.684,0,0,,2,5,.400,2,7,9,3,0,1,0,1,28,.660,.684,.000,.263,6.3,16.3,12.0,11.5,0.0,1.4,0.0,22.0,138,90,8.5
2,1985,Regular Season,1,1984-10-26,CHI,Chicago Bulls,Home,0,W,Ennis Whatley,37:00,5,8,.625,0,0,,1,1,1.000,1,3,4,10,2,0,4,2,11,.652,.625,.000,.125,3.3,7.3,5.6,31.3,2.4,0.0,32.2,13.6,110,89,1.9
3,1985,Regular Season,1,1984-10-26,CHI,Chicago Bulls,Home,0,W,Steve Johnson,35:00,9,14,.643,0,0,,0,1,.000,5,4,9,3,2,1,2,2,18,.623,.643,.000,.071,17.6,10.4,13.4,11.5,2.6,1.6,12.2,19.0,120,87,6.2
4,1985,Regular Season,1,1984-10-26,CHI,Chicago Bulls,Home,0,W,Caldwell Jones,29:00,0,5,.000,0,1,.000,0,0,,0,5,5,5,0,0,0,4,0,.000,.000,.200,.000,0.0,15.6,9.0,17.2,0.0,0.0,0.0,7.0,57,90,-3.2


In [99]:
mj_seasons.to_csv("../data/mj_bulls_seasons.csv", index = False)